#1 Successive Over-Relaxation (SOR) method

$$x^{(k)} = (1-\omega)x_i^{(k-1)} + \frac{\omega}{a_{ii}} \big[ \sum_{j=1}^{i-1} (-a_{ij}x^{(k)}_{j}) + \sum_{j=i+1}^{n} (-a_{ij}x^{(k-1)}_{j}) + b_i \big].$$

> Note that when $\omega = 1$, it becomes GS method.


##Task 1.1:  Implement SOR by modifying GS.

In [66]:
function SOR(A, b, x0, w, TOL, Nmax)
    n, n1 = size(A)
    x = zeros(n,1)
    for k = 1:Nmax
        for i = 1:n
            temp1 = 0
            for j = 1:i-1
                temp1 = temp1 + A[i,j] * x[j]
            end
            
            temp2 = 0
            for j = i+1:n
                temp2 = temp2 + A[i,j] * x0[j]
            end
            
            x[i] = (1 - w)*x0[i] + (w/A[i,i])*((-temp1) - temp2 + b[i])
        end
        res = norm(x - x0, Inf) # make sure calculating inf norm
        if res < TOL
            println("iterations: ", k)
            return x
        end
        x0[1:n] = x[1:n]
    end
    println("Reach the maximum iterations! Solution might be wrong")
end

SOR (generic function with 1 method)

##Task1.2: Test it on the following example.
$$
\begin{bmatrix}
4 & 3 & 0 \\
3 & 4 & -1 \\
0 & -1 & 4
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix}
=
\begin{bmatrix}
24\\
30\\
-24
\end{bmatrix},
$$ where the exact solution is $(3, 4, -5).$  Use $(1,1,1)$ as an initial iterate.  Run it with $w=$ 0.1, 1.25, 1.5, 2.1.  What do you observe? 

In [67]:
A = [
    4.0 3 0;
    3 4 -1;
    0 -1 4
]
b = [24.0; 30; -24]
x0 = [0.0; 0; 0]
w = 1.25
TOL = 1e-5
Nmax = 100
SOR(A, b, x0, w, TOL, Nmax)

iterations: 12

3x1 Array{Float64,2}:
  3.0
  4.0
 -5.0

##Task 1.3: Redo Section 7.3 #15 from HW3 by SOR method.  Can you find $\omega$ so that SOR is faster than GS?

fastest when w = 1.25

#2 Conjugate Gradient (CG) Method
CG is to solve $Ax = b$ for $x$, and $A$ needs to be SPD.

* $r_0 = b - Ax_0$
* $p_1 = r_0$
* Do k = 1, ..., n
  * $\displaystyle \alpha_k = \frac{r_{k-1}^T r_{k-1}}{p_k^T Ap_k}$
  * $\displaystyle x_k = x_{k-1} + \alpha_k p_k$
  * $\displaystyle r_k = r_{k-1} - \alpha_k Ap_k$
  * If $r_k$ is small, then return $x_k$
  * $\displaystyle \beta_k = \frac{r_{k}^T r_{k}}{r_{k-1}^T r_{k-1}}$
  * $\displaystyle p_k = r_k + \beta_k p_k$

In [68]:
function CG(A, b, x, TOL)
    r = b[:] - A[:,:]*x[:]
    p = zeros(size(r))
    p[:] = r[:]
    rsold = dot(r,r)
    for i = 1:length(b)
        Ap = A[:,:] * p[:]
        alpha = rsold / dot(p, Ap)
        x = x[:] + alpha * p[:]
        r = r - alpha * Ap
        rsnew = dot(r,r)
        if sqrt(rsnew) < TOL
            println("iterations: ", i)
            return x
        end
        p = r + (rsnew/rsold)*p[:]
        rsold = rsnew
    end
    println("iterations:", i)
    return x
end

CG (generic function with 1 method)

##Task 2.1: Test it on the example as in Task 1.2.  Compare the number of iterations it takes to converge.

In [69]:
A = [
    4.0 3 0;
    3 4 -1;
    0 -1 4
]
b = [24.0; 30; -24]
x0 = [0.0; 0; 0]
TOL = 1e-8
CG(A, b, x0, TOL)

iterations: 3

3-element Array{Float64,1}:
  3.0
  4.0
 -5.0

##Task 2.2: Redo Section 7.3 #16a from HW3 with CG.  How many iterations  do CG method take to converge?

In [70]:
function genTriDiag(n, l, d, r)
    #generate a tridiagonal matrix of size n by n with entries l, d, r.
    T = zeros(n,n)
    for i = 1:n
        T[i,i] = d
        if i+1 <= n
            T[i, i+1] = r
        end
        if i-1 >= 1
            T[i, i-1] = l
        end
    end
    return T
end

genTriDiag (generic function with 1 method)

In [71]:
n = 100
A = genTriDiag(n, -0.5, 1, -0.5)
b = zeros(n,1)
b[1] = 0.5
x0 = zeros(n,1)
CG(A, b, x0, 1e-8)


iterations: 100

100-element Array{Float64,1}:
 0.990099  
 0.980198  
 0.970297  
 0.960396  
 0.950495  
 0.940594  
 0.930693  
 0.920792  
 0.910891  
 0.90099   
 0.891089  
 0.881188  
 0.871287  
 ⋮         
 0.118812  
 0.108911  
 0.0990099 
 0.0891089 
 0.0792079 
 0.0693069 
 0.0594059 
 0.049505  
 0.039604  
 0.029703  
 0.019802  
 0.00990099

##Task 2.3(Optional): Implement the preconditioned CG (PCG) method as in the following link.
https://en.wikipedia.org/wiki/Conjugate_gradient_method#The_preconditioned_conjugate_gradient_method